In [ ]:
# You might want to install some useful Python packages to get started
# For example, here are several helpful packages to install 

from IPython.core.interactiveshell import InteractiveShell

# pretty print all cell's output and not just the last one
InteractiveShell.ast_node_interactivity = "all"

# From a terminal run `pip install numpy` than uncomment the next line
import numpy as np  # linear algebra

import time

import seaborn as sns

import matplotlib.pyplot as plt

# From a terminal run `pip install pandas` than uncomment the next line
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "/xprize" directory.
# For example, running this snippet (by clicking run or pressing Shift+Enter)
# will list all files under the input directory

allData = {}

paths = ['us-covid-data', 'us-county-health-rankings', 'us-mental-health-rankings', 'covid-deaths-us', 'mobility', 'lock-down']

for path in paths:

#     import os
#     for dirname, _, filenames in os.walk('./xprize/adrianwilliams/' + path):
#         for filename in filenames:
#             if '.ipynb' not in dirname:            
#                 print(os.path.join(dirname, filename))

    import glob

    allFiles = glob.glob('./xprize/adrianwilliams/' + path + '/*.csv')

    allData[path] = {}
    
    allData[path +'_keys'] = []

    for iFile in range(len(allFiles)):

        fileKey = allFiles[iFile].split('/')[-1].replace(' ','_')[:-4]

        file = allFiles[iFile]
        cols = !head -n 1 $file

        allData[path][fileKey] = pd.read_csv(allFiles[iFile], sep=',', header=0)#, names=cols[0].split(','))

        allData[path +'_keys'].append(fileKey)

#         print(allData[path][fileKey])           

In [ ]:
#get state name to state code mapper
statePostalCodeMapper = pd.read_csv('./xprize/adrianwilliams/state_codes.csv')

# statePostalCodeMapper

In [ ]:
#get covid case and death rate (aggregate daily values over full 2 week time range) 3/4/2020 - 3/18/2020 
# allData['us-covid-data'][allData['us-covid-data_keys'][0]]['date'].unique()
# ourUsCovidData = allData['us-covid-data'][allData['us-covid-data_keys'][0]].groupby('state').agg('sum')[['positive','negative','pending','death','total']]
# ourUsCovidData

In [ ]:
rawTable = allData['us-county-health-rankings'][allData['us-county-health-rankings_keys'][0]][['state', 'county', 'average_number_of_physically_unhealthy_days', 'average_number_of_mentally_unhealthy_days']].copy()

rawTable['unhealthy_days'] = rawTable['average_number_of_physically_unhealthy_days'] + rawTable['average_number_of_mentally_unhealthy_days']

ourUsHealthData = rawTable.groupby(['state']).agg('mean').sort_values('unhealthy_days', ascending=False)[['average_number_of_physically_unhealthy_days', 'average_number_of_mentally_unhealthy_days']]

fig = plt.figure(figsize=(12,8))
ax = fig.add_subplot(1, 1, 1)
ourUsHealthData.plot.bar(stacked=True, ax=ax)
ax.set_title("pre-COVID health indicators by state (2019)")
print(ax)
plt.gcf().subplots_adjust(bottom=0.15)
plt.tight_layout()
fig.savefig('pre-COVID health indicators by state (2019).png')

del ourUsHealthData

In [ ]:
rawTable = allData['us-county-health-rankings'][allData['us-county-health-rankings_keys'][0]][['state', 'county', 'average_number_of_physically_unhealthy_days', 'average_number_of_mentally_unhealthy_days']].copy()

ourUsHealthData = rawTable.groupby(['state'], as_index=False).agg('mean')

ourUsHealthData['average_number_of_unhealthy_days'] = ourUsHealthData['average_number_of_physically_unhealthy_days'] + ourUsHealthData['average_number_of_mentally_unhealthy_days']
ourUsHealthData.columns

In [ ]:
#get poplulation data by state

#get data from us health rankings (values seem low, are these adults only?)
# popData = allData['us-county-health-rankings'][allData['us-county-health-rankings_keys'][0]][['state', 'county', 'population']]
# ourAdultPopulationData = popData[popData['county'].isnull()].groupby('state').agg('sum')
# ourAdultPopulationData

#estimated state populations by census projections
ourTotalPopulationData = pd.read_csv('./xprize/adrianwilliams/total_population.csv', names=['state', 'population'])
ourTotalPopulationData['state'] = ourTotalPopulationData['state'].str.lstrip()

In [ ]:
#get mental health ranking data

our_measures = ['All Determinants', 'All Outcomes', 'Frequent Mental Distress', 'Frequent Physical Distress', 'Depression', 'Violent Crime', 'Uninsured', 'Suicide', 'Seat Belt Use', 'Premature Death', 'Physical inactivity']

#choosing mulitple indicators here will cause duplicates in final totalTable join

# our_measures = ['All Outcomes']
allMentalHealthRankings = allData['us-mental-health-rankings'][allData['us-mental-health-rankings_keys'][0]]
ourMentalHealthRankings = allMentalHealthRankings[allMentalHealthRankings['Measure Name'].isin(our_measures)][['Measure Name', 'State Name', 'Rank', 'Source Year']]
ourMentalHealthRankings = ourMentalHealthRankings.rename(columns={'State Name' : 'state'})

In [ ]:
ourMentalHealthRankings.columns

In [ ]:
#get us covid death data by state

deaths = allData['covid-deaths-us'][allData['covid-deaths-us_keys'][0]]
deaths['Total']= deaths.iloc[:, -166:-1].sum(axis=1)
deaths['State'] = deaths['State'].replace(list(statePostalCodeMapper['code']), list(statePostalCodeMapper['state']))
ourCovidDeathData = deaths[['County Name', 'State', 'Total']].groupby('State', as_index=False).agg('sum')
ourCovidDeathData = ourCovidDeathData.rename(columns={'State' : 'state'})
ourCovidDeathData.columns

In [ ]:
# get US mobility data
globalMobility = allData['mobility'][allData['mobility_keys'][0]]
usMobility = globalMobility[globalMobility['country_region']=='United States']
usMobility = usMobility[usMobility['sub_region_1'].notnull()]
usMobility = usMobility[usMobility['sub_region_2'].notnull()].groupby(['sub_region_1', 'sub_region_2'],as_index=False).agg('mean').groupby('sub_region_1',as_index=False).agg('mean')
usMobility = usMobility.rename(columns={'sub_region_1' : 'state'})
usMobility.columns

In [ ]:
# # get US lock-down data
# lockDownData = allData['lock-down'][allData['lock-down_keys'][0]].copy()
# lockDownData = lockDownData.replace(['Stay at home'],1)
# lockDownData = lockDownData.replace(['Shelter in place'],2)
# lockDownData = lockDownData.replace(['Curfew'],3).groupby('State').agg('mean')
# lockDownData

In [ ]:
#join ALL tables by state (ourCovidDeathData, ourUsHealthData, ourTotalPopulationData, ourMentalHealthRankings, usMobility)
totalTable = ourCovidDeathData.merge(ourTotalPopulationData, on='state',how='inner')
#estimate death rate per-capita
totalTable = totalTable.rename(columns={'Total' : 'total_covid_deaths'})
totalTable['covid_death_rate'] = totalTable['total_covid_deaths']/totalTable['population']

totalTable = totalTable.merge(ourUsHealthData, on='state',how='inner').merge(usMobility, on='state',how='inner').merge(ourMentalHealthRankings, on='state',how='inner')
totalTable.columns


In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(1, 1, 1)

totalTable.plot(kind='scatter', x='covid_death_rate', y='retail_and_recreation_percent_change_from_baseline', color='r', ax=ax1)    
x=totalTable.covid_death_rate
y=totalTable.retail_and_recreation_percent_change_from_baseline
m, b = np.polyfit(x, y, 1)
# A,B,C = np.polyfit(x, y, 2)
ax1.plot(x, m*x + b)
# ax1.plot(x, A*x**2 + B*x + C)
ax1.set_title('change in retail/recreation vs COVID death rate per state')
print(ax1)
fig.savefig('change in retail-recreation vs COVID death rate per state.png')

In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(1, 1, 1)

totalTable.plot(kind='scatter', x='covid_death_rate', y='grocery_and_pharmacy_percent_change_from_baseline', color='r', ax=ax1)    
x=totalTable.covid_death_rate
y=totalTable.grocery_and_pharmacy_percent_change_from_baseline
m, b = np.polyfit(x, y, 1)
# A,B,C = np.polyfit(x, y, 2)
ax1.plot(x, m*x + b)
# ax1.plot(x, A*x**2 + B*x + C)
ax1.set_title('change in grocery/pharmacy vs COVID death rate per state')
print(ax1)
fig.savefig('change in grocery-pharmacy vs COVID death rate per state.png')

In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(1, 1, 1)

totalTable.plot(kind='scatter', x='covid_death_rate', y='parks_percent_change_from_baseline', color='r', ax=ax1)    
x=totalTable.covid_death_rate
y=totalTable.parks_percent_change_from_baseline
m, b = np.polyfit(x, y, 1)
# A,B,C = np.polyfit(x, y, 2)
ax1.plot(x, m*x + b)
# ax1.plot(x, A*x**2 + B*x + C)
ax1.set_title('change in parks vs COVID death rate per state')
print(ax1)
fig.savefig('change in parks vs COVID death rate per state.png')

In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(1, 1, 1)

totalTable.plot(kind='scatter', x='covid_death_rate', y='transit_stations_percent_change_from_baseline', color='r', ax=ax1)    
x=totalTable.covid_death_rate
y=totalTable.transit_stations_percent_change_from_baseline
m, b = np.polyfit(x, y, 1)
# A,B,C = np.polyfit(x, y, 2)
ax1.plot(x, m*x + b)
# ax1.plot(x, A*x**2 + B*x + C)
ax1.set_title('change in transit vs COVID death rate per state')
print(ax1)
fig.savefig('change in transit vs COVID death rate per state.png')

In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(1, 1, 1)

totalTable.plot(kind='scatter', x='covid_death_rate', y='workplaces_percent_change_from_baseline', color='r', ax=ax1)    
x=totalTable.covid_death_rate
y=totalTable.workplaces_percent_change_from_baseline
m, b = np.polyfit(x, y, 1)
# A,B,C = np.polyfit(x, y, 2)
ax1.plot(x, m*x + b)
# ax1.plot(x, A*x**2 + B*x + C)
ax1.set_title('change in workplace commute vs COVID death rate per state')
print(ax1)
fig.savefig('change in workplace commute vs COVID death rate per state.png')

In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(1, 1, 1)

totalTable.plot(kind='scatter', x='covid_death_rate', y='residential_percent_change_from_baseline', color='r', ax=ax1)    
x=totalTable.covid_death_rate
y=totalTable.residential_percent_change_from_baseline
m, b = np.polyfit(x, y, 1)
# A,B,C = np.polyfit(x, y, 2)
ax1.plot(x, m*x + b)
# ax1.plot(x, A*x**2 + B*x + C)
ax1.set_title('change in staying home vs COVID death rate per state')
print(ax1)
fig.savefig('change in staying home vs COVID death rate per state.png')

In [ ]:
#take aways - The data suggest that as covid death rates rose, 
#people stayed home compared to going to work, transit or general recreation 
#however we have found that more people are going to parks and grocery/pharamcy shopping stays about the same until death rate exceed 3-5%
#(this makes sense since it is outdoors and people can still distance)

In [ ]:
totalTable.columns

In [ ]:
our_measures

In [ ]:
tempTable = totalTable[totalTable['Measure Name']=='All Determinants']

fig = plt.figure()
ax1 = fig.add_subplot(1, 1, 1)

tempTable.plot(kind='scatter', x='Rank', y='covid_death_rate', color='r', ax=ax1)    
x=tempTable.Rank
y=tempTable.covid_death_rate
m, b = np.polyfit(x, y, 1)
# A,B,C = np.polyfit(x, y, 2)
ax1.plot(x, m*x + b)
# ax1.plot(x, A*x**2 + B*x + C)
ax1.set_title('COVID death rate vs All Determinants Healthcare Ranking')
print(ax1)
fig.savefig('COVID death rate vs All Determinants Healthcare Ranking.png')

In [ ]:
tempTable = totalTable[totalTable['Measure Name']=='All Outcomes']

fig = plt.figure()
ax1 = fig.add_subplot(1, 1, 1)

tempTable.plot(kind='scatter', x='Rank', y='covid_death_rate', color='r', ax=ax1)    
x=tempTable.Rank
y=tempTable.covid_death_rate
m, b = np.polyfit(x, y, 1)
# A,B,C = np.polyfit(x, y, 2)
ax1.plot(x, m*x + b)
# ax1.plot(x, A*x**2 + B*x + C)
ax1.set_title('COVID death rate vs All Outcomes Healthcare Ranking')
print(ax1)
fig.savefig('COVID death rate vs All Outcomes Healthcare Ranking.png')

In [ ]:
tempTable = totalTable[totalTable['Measure Name']=='Seat Belt Use']

fig = plt.figure()
ax1 = fig.add_subplot(1, 1, 1)

tempTable.plot(kind='scatter', x='Rank', y='covid_death_rate', color='r', ax=ax1)    
x=tempTable.Rank
y=tempTable.covid_death_rate
m, b = np.polyfit(x, y, 1)
# A,B,C = np.polyfit(x, y, 2)
ax1.plot(x, m*x + b)
# ax1.plot(x, A*x**2 + B*x + C)
ax1.set_title('COVID death rate vs Seat Belt Use Ranking')
print(ax1)
fig.savefig('COVID death rate vs Seat Belt Use Ranking.png')

In [ ]:
#duplicate values account for muliple rankings (multi-schema table)
totalTable